<a href="https://colab.research.google.com/github/christianwarmuth/transformer_adapter_bias_evaluation/blob/main/colab/Adapter_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Evaluating Adapter 

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)
#drive.mount("/content/gdrive/MyDrive/master_hpi/NLP_Project/code/", force_remount=True)

Mounted at /content/gdrive/


In [22]:
import sys
sys.path.append('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/')



In [23]:
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"

In [24]:
!pip install adapter-transformers
!pip install datasets
!pip install colorama

In [25]:
import pandas as pd
import dataloader
from collections import defaultdict 

# Dataset Loading - Stereoset

In [26]:

print(path+"dev.json")

/content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json


In [27]:
stereoset = dataloader.StereoSet(path+"dev.json")

In [28]:
stereoset

In [29]:
intersentence_examples = stereoset.get_intersentence_examples() 
intrasentence_examples = stereoset.get_intrasentence_examples()

In [30]:
intrasentence_examples[1].sentences[1].gold_label

'anti-stereotype'

In [31]:
id2term = {}
id2gold = {}
id2score = {}
example2sent = {}
domain2example = {"intersentence": defaultdict(lambda: []), "intrasentence": defaultdict(lambda: [])}

for example in intrasentence_examples:
  for sentence in example.sentences:
    id2term[sentence.ID] = example.target
    id2gold[sentence.ID] = sentence.gold_label
    example2sent[(example.ID, sentence.gold_label)] = sentence.ID
    domain2example['intrasentence'][example.bias_type].append(example)

for example in intersentence_examples:
  for sentence in example.sentences:
    id2term[sentence.ID] = example.target
    id2gold[sentence.ID] = sentence.gold_label
    example2sent[(example.ID, sentence.gold_label)] = sentence.ID
    domain2example['intersentence'][example.bias_type].append(example)

In [33]:
# When predictions are already done
#for sent in self.predictions.get('intrasentence', []) + self.predictions.get('intersentence', []):
            self.id2score[sent['id']] = sent['score']

IndentationError: ignored

In [15]:
domain2example

{'intersentence': defaultdict(<function __main__.<lambda>>,
             {'gender': [<dataloader.IntersentenceExample at 0x7fbd69075a10>,
              'profession': [<dataloader.IntersentenceExample at 0x7fbd69078b10>,
               ...],
              'race': [<dataloader.IntersentenceExample at 0x7fbd690f2f50>,
               ...],
              'religion': [<dataloader.IntersentenceExample at 0x7fbd68e8f250>,
               <dataloader.IntersentenceExample at 0x7fbd6881afd0>]}),
 'intrasentence': defaultdict(<function __main__.<lambda>>,
             {'gender': [<dataloader.IntrasentenceExample at 0x7fbd6997f310>,
              'profession': [<dataloader.IntrasentenceExample at 0x7fbd6997c310>,
               ...],
              'race': [<dataloader.IntrasentenceExample at 0x7fbd6997cad0>,
               ...],
              'religion': [<dataloader.IntrasentenceExample at 0x7fbd697a5d10>,
               <dataloader.IntrasentenceExample at 0x7fbd690d5850>]})}

In [46]:
from eval_template import *
import dataloader


class Args:
  no_cuda = False
  input_file = '/content/dev.json'
  output_file = '/content/predictions.json'
  max_seq_length = 128
  batch_size = 1

args=Args()

#class input_args:
#  def __init__(self, no_cuda, input_file, output_file, max_seq_length):
#    self.no_cuda = no_cuda
 #   self.input_file = input_file
 #   self.output_file = output_file
#    self.max_seq_length = max_seq_length

#args = input_args(False, "/content/dev.json", "/content/predictions.json", 128)
#args = {#
#    "no_cuda": False, 
#    "input_file": "/content/dev.json", 
#    "output_file":"/content/predictions.json"
#}

evaluator = BiasEvaluator(**vars(args))
results = evaluator.evaluate()

if args.output_file is not None:
  output_file = args.output_file
else:
  output_file = f"predictions_SentimentModel.json"

  output_file = os.path.join(args.output_dir, output_file)
with open(output_file, "w+") as f:
  json.dump(results, f, indent=2)


Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: pretrained_models/SentimentBert.pth
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2107: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  FutureWarning,


Maximum sequence length found: -inf
Arrived 0 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Number of parameters: 109,483,778
Arrived 1


RuntimeError: ignored

In [42]:
!wget -P pretrained_models http://moinnadeem.com/stereoset/pretrained_models/SentimentBert.pth


--2021-06-20 17:20:02--  http://moinnadeem.com/stereoset/pretrained_models/SentimentBert.pth
Resolving moinnadeem.com (moinnadeem.com)... 172.67.148.110, 104.21.87.237, 2606:4700:3035::ac43:946e, ...
Connecting to moinnadeem.com (moinnadeem.com)|172.67.148.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 437988350 (418M) [application/octet-stream]
Saving to: ‘pretrained_models/SentimentBert.pth’

SentimentBert.pth   100%[===================>] 417.70M  13.8MB/s    in 31s     

2021-06-20 17:20:33 (13.5 MB/s) - ‘pretrained_models/SentimentBert.pth’ saved [437988350/437988350]



In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
class Args:
  no_cuda = False
  input_file = '/content/sample_data/dev.json'
  output_file = '/content/sample_data/predictions.json'

args=Args()

In [ ]:
print(vars(args))